In [1]:
DATA_NAME = 'pastis-full' 
TRANSFORM = 'fourier'
CHANNEL = 'gray'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_gray_fourier_full_pastis.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10,11,...,24,25,26,27,28,29,30,31,32,33
2,324.30,-0.02,0.00,-0.01,-0.00,0.01,0.00,-0.00,0.00,-0.00,...,0.00,0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,0.0,-0.0
3,-0.02,242.65,0.00,0.00,0.01,0.01,-0.00,-0.00,0.00,-0.00,...,-0.00,-0.00,-0.00,-0.00,-0.00,0.00,0.00,0.00,-0.0,-0.0
4,0.00,0.00,203.17,0.01,-0.00,-0.00,0.00,-0.00,0.00,0.00,...,0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.00,0.0,0.0
5,-0.01,0.00,0.01,157.32,0.01,-0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,-0.00,-0.00,-0.00,0.00,-0.00,0.00,-0.0,0.0
6,-0.00,0.01,-0.00,0.01,125.52,0.00,0.00,-0.00,-0.00,-0.00,...,-0.00,0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.00,0.0,0.0
7,0.01,0.01,-0.00,-0.00,0.00,96.68,0.00,-0.00,0.00,0.00,...,-0.00,-0.00,-0.00,-0.00,0.00,0.00,-0.00,-0.00,-0.0,0.0
8,0.00,-0.00,0.00,0.00,0.00,0.00,77.35,0.00,0.00,0.00,...,-0.00,0.00,0.00,0.00,-0.00,-0.00,-0.00,0.00,-0.0,-0.0
9,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.00,62.03,-0.00,0.00,...,0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.00,0.0,-0.0
10,0.00,0.00,0.00,0.00,-0.00,0.00,0.00,-0.00,49.08,0.00,...,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.0,0.0
11,-0.00,-0.00,0.00,0.00,-0.00,0.00,0.00,0.00,0.00,38.99,...,0.00,-0.00,0.00,-0.00,-0.00,0.00,-0.00,0.00,-0.0,-0.0


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10,11,...,24,25,26,27,28,29,30,31,32,33
2,1.00000,-0.00007,0.00002,-0.00003,-0.00002,0.00003,0.00003,-0.00003,0.00002,-0.00000,...,0.00002,0.00001,0.00002,-0.00002,-0.00000,-0.00000,0.00002,0.00001,0.00002,-0.00000
3,-0.00007,1.00000,0.00000,0.00002,0.00003,0.00005,-0.00002,-0.00003,0.00003,-0.00000,...,-0.00002,-0.00002,-0.00002,-0.00001,-0.00001,0.00000,0.00000,0.00001,-0.00000,-0.00001
4,0.00002,0.00000,1.00000,0.00006,-0.00003,-0.00001,0.00002,-0.00001,0.00001,0.00000,...,0.00000,-0.00000,-0.00001,-0.00002,0.00001,-0.00001,-0.00000,0.00001,0.00000,0.00001
5,-0.00003,0.00002,0.00006,1.00000,0.00004,-0.00002,0.00002,0.00005,0.00001,0.00000,...,0.00001,0.00002,-0.00001,-0.00002,-0.00002,0.00001,-0.00000,0.00001,-0.00000,0.00000
6,-0.00002,0.00003,-0.00003,0.00004,1.00000,0.00000,0.00003,-0.00001,-0.00002,-0.00000,...,-0.00000,0.00001,-0.00000,-0.00001,0.00001,-0.00000,-0.00000,0.00001,0.00000,0.00000
7,0.00003,0.00005,-0.00001,-0.00002,0.00000,1.00000,0.00002,-0.00000,0.00001,0.00000,...,-0.00004,-0.00000,-0.00000,-0.00001,0.00002,0.00001,-0.00003,-0.00002,-0.00001,0.00002
8,0.00003,-0.00002,0.00002,0.00002,0.00003,0.00002,1.00000,0.00007,0.00004,0.00000,...,-0.00003,0.00000,0.00001,0.00000,-0.00003,-0.00001,-0.00000,0.00000,-0.00001,-0.00001
9,-0.00003,-0.00003,-0.00001,0.00005,-0.00001,-0.00000,0.00007,1.00000,-0.00007,0.00001,...,0.00000,-0.00004,-0.00000,-0.00001,-0.00002,-0.00001,-0.00000,0.00000,0.00001,-0.00000
10,0.00002,0.00003,0.00001,0.00001,-0.00002,0.00001,0.00004,-0.00007,1.00000,0.00005,...,0.00000,-0.00000,0.00001,-0.00003,0.00001,-0.00000,0.00001,-0.00000,0.00000,0.00000
11,-0.00000,-0.00000,0.00000,0.00000,-0.00000,0.00000,0.00000,0.00001,0.00005,1.00000,...,0.00001,-0.00002,0.00001,-0.00001,-0.00002,0.00001,-0.00002,0.00000,-0.00001,-0.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.045851825145705634

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[3.29525944e+03 1.84346081e+03 1.28762217e+03 7.71742611e+02
 4.90288625e+02 2.90798097e+02 1.85777481e+02 1.19225209e+02
 7.45359869e+01 4.69431082e+01 2.87726539e+01 1.93076775e+01
 1.05457275e+01 6.82711525e+00 4.04128501e+00 2.35723578e+00
 1.33322710e+00 7.42597895e-01 3.99547972e-01 2.21674412e-01
 1.10088859e-01 5.30223198e-02 2.45104089e-02 1.10724248e-02
 4.37242914e-03 1.57285767e-03 5.24021026e-04 1.53502302e-04
 1.48047761e-05 1.80487948e-06 1.18686973e-07 6.49872750e-28]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,0.997757,-0.052354,-0.030987,-0.018933,-0.013526,-0.009659,-0.007457,-0.005882,-0.004564,-0.003603,...,-0.000082,-0.000056,-0.000035,-0.000022,-0.000013,-0.000007,-0.000002,-7.815984e-07,-1.940586e-07,-5.937920e-08
1,0.048039,0.993651,-0.090663,-0.034359,-0.021429,-0.014267,-0.010724,-0.008264,-0.006331,-0.004966,...,-0.000115,-0.000078,-0.000049,-0.000030,-0.000017,-0.000010,-0.000003,-1.086415e-06,-3.444649e-07,-9.427492e-08
2,0.033026,0.085123,0.992346,-0.068713,-0.034604,-0.021037,-0.015144,-0.011449,-0.008690,-0.006754,...,-0.000153,-0.000104,-0.000066,-0.000040,-0.000023,-0.000013,-0.000004,-1.423559e-06,-4.336134e-07,-8.969242e-08
3,0.020804,0.036083,0.060668,0.992938,-0.079763,-0.034544,-0.022407,-0.016025,-0.011860,-0.009054,...,-0.000198,-0.000134,-0.000086,-0.000052,-0.000031,-0.000017,-0.000005,-1.884607e-06,-5.968225e-07,-1.305822e-07
4,0.015857,0.024356,0.034254,0.072189,0.992428,-0.072554,-0.036803,-0.024014,-0.016872,-0.012520,...,-0.000265,-0.000179,-0.000114,-0.000069,-0.000040,-0.000023,-0.000007,-2.563459e-06,-7.520169e-07,-1.762946e-07
5,0.011481,0.016541,0.021489,0.034138,0.063513,0.991637,-0.086908,-0.040959,-0.025329,-0.017741,...,-0.000350,-0.000235,-0.000149,-0.000090,-0.000052,-0.000029,-0.000010,-3.618702e-06,-1.050212e-06,-1.923414e-07
6,0.009317,0.013085,0.016355,0.023693,0.035889,0.077043,0.989911,-0.092533,-0.041835,-0.026333,...,-0.000460,-0.000308,-0.000195,-0.000118,-0.000071,-0.000039,-0.000012,-4.509413e-06,-1.411583e-06,-3.205060e-07
7,0.007729,0.010632,0.013051,0.017981,0.025130,0.041289,0.081343,0.989173,-0.092346,-0.043224,...,-0.000599,-0.000410,-0.000257,-0.000156,-0.000092,-0.000051,-0.000016,-5.933257e-06,-1.703784e-06,-4.031281e-07
8,0.006226,0.008467,0.010303,0.013859,0.018488,0.027165,0.041567,0.080135,0.988587,-0.097313,...,-0.000786,-0.000533,-0.000337,-0.000206,-0.000119,-0.000067,-0.000021,-7.856966e-06,-2.283004e-06,-5.143538e-07
9,0.005148,0.006958,0.008398,0.011117,0.014456,0.020163,0.028113,0.042690,0.084536,0.987911,...,-0.001037,-0.000706,-0.000445,-0.000270,-0.000159,-0.000088,-0.000028,-1.027070e-05,-3.114111e-06,-7.078934e-07


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0022427023937773294,
 0.00634872094901906,
 0.007654121288180882,
 0.007061705548953134,
 0.007571934484200682,
 0.008362878651937744,
 0.010089255280033904,
 0.01082664293902913,
 0.011413037583418162,
 0.01208927834848672,
 0.014737060247841272,
 0.014736639725440925,
 0.014343306167984404,
 0.01687047836203792,
 0.015998655224231828,
 0.016334962025659094,
 0.01687953868934211,
 0.017674795958785183,
 0.019679656078380847,
 0.020597345826048108,
 0.019682003963947703,
 0.02076691856002233,
 0.022700336694685186,
 0.023550475346947652,
 0.023142836004930056,
 0.024907973026793373,
 0.027840552692937504,
 0.0215193661643871,
 0.0157916938781556,
 0.02344685668852886,
 0.039165155435603616,
 0.027648210819419705]